In [15]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd

# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
try:
    from config import (consumer_key, 
                        consumer_secret, 
                        access_token, 
                        access_token_secret)
except:
    consumer_key = os.environ["CONSUMER_KEY"]
    consumer_secret = os.environ["CONSUMER_SECRET"]
    access_token = os.environ["ACCESS_TOKEN"]
    access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
    
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [16]:
# Target Search Term
target_terms = ("@SouthwestAir", "@AmericanAir", "@SpiritAirlines",
                "@Virginatlantic", "@Delta", "@AlaskaAir", "@KLM", "@FlyFrontier")

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# List to hold sentiment
results_list = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        public_tweets = api.search(target, 
                               count=100, 
                               result_type="recent", 
                               max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in public_tweets["statuses"]: #only gets the keys that are nested within statuses.

            # Use filters to check if user meets conditions
            if (tweet["user"]["statuses_count"] > min_tweets and
                tweet["user"]["statuses_count"] < max_tweets and
                tweet["user"]["friends_count"] < max_following and
                tweet["user"]["followers_count"] < max_followers and
                tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                    results = analyzer.polarity_scores(tweet["text"])

                # Add each value to the appropriate list
                    compound_list.append(results["compound"])
                    positive_list.append(results["pos"])
                    negative_list.append(results["neg"])
                    neutral_list.append(results["neu"])
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1
            oldest_tweeter = tweet["created_at"]

    # Create a dictionary of the Average Sentiments
    airline_results = {"Airline":target,
                      "Compound Score":np.mean(compound_list),
                      "Pos Score":np.mean(positive_list),
                      "Neg Score":np.mean(negative_list),
                      "Neut Score":np.mean(neutral_list),
                      "Oldest Tweet":oldest_tweeter}

    # Print the Sentiments
    print(airline_results)
    print("-----")
    
    # Append the dictionary to results_list
    results_list.append(airline_results)

{'Airline': '@SouthwestAir', 'Compound Score': 0.1622003076923077, 'Pos Score': 0.13137076923076924, 'Neg Score': 0.05215538461538461, 'Neut Score': 0.8164676923076922, 'Oldest Tweet': 'Fri Jul 06 17:00:39 +0000 2018'}
-----
{'Airline': '@AmericanAir', 'Compound Score': 0.015757937806873977, 'Pos Score': 0.08579214402618657, 'Neg Score': 0.07985924713584287, 'Neut Score': 0.8343502454991817, 'Oldest Tweet': 'Sat Jul 07 02:51:54 +0000 2018'}
-----
{'Airline': '@SpiritAirlines', 'Compound Score': -0.01317603174603175, 'Pos Score': 0.09164761904761905, 'Neg Score': 0.09447777777777779, 'Neut Score': 0.8138873015873016, 'Oldest Tweet': 'Tue Jul 03 01:55:44 +0000 2018'}
-----
{'Airline': '@Virginatlantic', 'Compound Score': 0.18520114942528734, 'Pos Score': 0.1284367816091954, 'Neg Score': 0.04170306513409961, 'Neut Score': 0.8298563218390804, 'Oldest Tweet': 'Thu Jul 05 06:21:49 +0000 2018'}
-----
{'Airline': '@Delta', 'Compound Score': 0.07331712439418415, 'Pos Score': 0.10621809369951533

In [17]:
# Create a DataFrame using results_list and display
Total_results = pd.DataFrame(results_list).set_index("Airline").round(3)
Total_results = Total_results.sort_values("Compound Score", ascending=True)
Total_results

Compound Score  Neg Score  Neut Score  \
Airline                                                  
@FlyFrontier             -0.024      0.090       0.834   
@SpiritAirlines          -0.013      0.094       0.814   
@AmericanAir              0.016      0.080       0.834   
@KLM                      0.072      0.052       0.853   
@Delta                    0.073      0.070       0.824   
@SouthwestAir             0.162      0.052       0.816   
@Virginatlantic           0.185      0.042       0.830   
@AlaskaAir                0.347      0.067       0.774   

                                   Oldest Tweet  Pos Score  
Airline                                                     
@FlyFrontier     Tue Jul 03 05:43:57 +0000 2018      0.076  
@SpiritAirlines  Tue Jul 03 01:55:44 +0000 2018      0.092  
@AmericanAir     Sat Jul 07 02:51:54 +0000 2018      0.086  
@KLM             Thu Jul 05 20:26:51 +0000 2018      0.095  
@Delta           Fri Jul 06 22:14:28 +0000 2018      0.106  
@SouthwestAir    Fri Jul 06 17:00:39 +0000 2018      0.131  
@Virginatlantic  Thu Jul 05 06:21:49 +0000 2018      0.128  
@AlaskaAir       Fri Jul 06 02:17:28 +0000 2018      0.159